# TFM

##1. Configuacion

In [25]:
# Instalar librerias
!pip install pyeph > /dev/null 2>&1
!pip install scikit-learn openpyxl > /dev/null 2>&1

In [26]:
# Importar librerias
import pyeph
import pandas as pd
import numpy as np

from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC

##2. Extraer bases de datos
Se utiliza la libreria [pyeph](https://pypi.org/project/pyeph/)

In [27]:
# Diccionarios para guardar las bases
bases_individuales = {}
bases_hogar = {}

# Descargar datos para cada año y trimestre
for ano in range(2023, 2024):  # Años 2023 a 2023
    for trimestre in range(1, 5):  # Trimestres 1 a 4
        clave = f"{ano}T{trimestre}"

        # Base individual
        try:
            df_ind = pyeph.obtener(data="eph", ano=ano, periodo=trimestre, tipo_base="individual")
            bases_individuales[clave] = df_ind
            print(f"✓ Base individual {clave} descargada")
        except Exception as e:
            print(f"✗ Error al descargar base individual {clave}: {e}")

        # Base hogar
        try:
            df_hogar = pyeph.obtener(data="eph", ano=ano, periodo=trimestre, tipo_base="hogar")
            bases_hogar[clave] = df_hogar
            print(f"✓ Base hogar {clave} descargada")
        except Exception as e:
            print(f"✗ Error al descargar base hogar {clave}: {e}")

Obtenido con exito: base_individual_2023T1.zip 
✓ Base individual 2023T1 descargada
Obtenido con exito: base_hogar_2023T1.zip 
✓ Base hogar 2023T1 descargada
Obtenido con exito: base_individual_2023T2.zip 
✓ Base individual 2023T2 descargada
Obtenido con exito: base_hogar_2023T2.zip 
✓ Base hogar 2023T2 descargada
Obtenido con exito: base_individual_2023T3.zip 
✓ Base individual 2023T3 descargada
Obtenido con exito: base_hogar_2023T3.zip 
✓ Base hogar 2023T3 descargada
Obtenido con exito: base_individual_2023T4.zip 
✓ Base individual 2023T4 descargada
Obtenido con exito: base_hogar_2023T4.zip 
✓ Base hogar 2023T4 descargada


##3. Procesar datos

###3.1 Crear bases

In [28]:
# Definir la clave correspondiente al cuarto trimestre de 2023
clave = "2023T4"

# Lista exhaustiva de variables relevantes a nivel hogar
variables_hogar_interes = [
    # Identificación y ubicación
    'CODUSU', 'NRO_HOGAR', 'ANO4', 'TRIMESTRE',
    'REGION', 'MAS_500', 'AGLOMERADO',

    # Ponderadores
    'PONDERA',   # Simple
    'PONDIH',    # Corregido por ingresos

    # Composición del hogar
    'IX_TOT', 'IX_MEN10', 'IX_MAYEQ10',

    # Ingresos y deciles
    'ITF', 'IPCF', 'DECIFR', 'IDECIFR', 'RDECIFR',
    'GDECIFR', 'PDECIFR', 'ADECIFR',
    'DECCFR', 'IDECCFR', 'RDECCFR',
    'GDECCFR', 'PDECCFR', 'ADECCFR',

    # Características habitacionales
    'IV1', 'IV2', 'IV3', 'IV4', 'IV5',
    'IV6', 'IV7', 'IV8', 'IV9', 'IV10', 'IV11',
    'IV12_1', 'IV12_2', 'IV12_3',

    # Ambientes y espacios complementarios
    'II1', 'II2', 'II3', 'II3_1', 'II4_1', 'II4_2', 'II4_3',
    'II5', 'II5_1', 'II6', 'II6_1', 'II7', 'II8', 'II9',

    # Estrategias de ingresos y subsistencia
    'V1', 'V2', 'V21', 'V22', 'V3', 'V4',
    'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
    'V11', 'V12', 'V13', 'V14', 'V15', 'V16',
    'V17', 'V18', 'V19_A', 'V19_B',

    # Organización del hogar: tareas domésticas
    'VII1_1', 'VII1_2',
    'VII2_1', 'VII2_2', 'VII2_3', 'VII2_4'
]

# Crear el nuevo DataFrame con solo las columnas seleccionadas
df_hogar_interes = bases_hogar[clave][variables_hogar_interes].copy()

# Mostrar primeras filas
df_hogar_interes.head()

,CODUSU,NRO_HOGAR,ANO4,TRIMESTRE,REGION,MAS_500,AGLOMERADO,PONDERA,PONDIH,IX_TOT,...,V17,V18,V19_A,V19_B,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,TQRMNORSQHKMKUCDEIGED00791045,2,2023,4,43,N,30,107,128,3,...,2,2,2,2,1,0,98,0,0,0
1,TQRMNOPVSHJMLNCDEIGED00801502,1,2023,4,43,N,30,137,0,2,...,2,2,2,2,1,0,2,0,0,0
2,TQRMNORTYHMLKPCDEHJGH00810920,1,2023,4,40,S,23,356,329,4,...,2,2,2,2,2,0,1,0,0,0
3,TQRMNOQTWHKLKQCDEHJGH00790284,2,2023,4,40,S,23,299,320,3,...,2,2,2,2,1,0,2,0,0,0
4,TQRMNORQVHLLKRCDEHJGH00853836,1,2023,4,40,S,23,221,193,2,...,2,2,2,2,1,0,96,0,0,0


In [29]:
# Definir la clave correspondiente al cuarto trimestre de 2023
clave = "2023T4"

# Variables de interés a nivel individuo (extensiva pero relevante)
variables_individuo_interes = [
    # Identificación y estructura
    'CODUSU', 'ANO4', 'TRIMESTRE', 'NRO_HOGAR', 'COMPONENTE',
    'REGION', 'MAS_500', 'AGLOMERADO', 'PONDERA', 'PONDIH',

    # Demográficas
    'CH03', 'CH04', 'CH05', 'CH06', 'CH07',

    # Cobertura de salud y alfabetización
    'CH08', 'CH09',

    # Variables educativas clave
    'CH10', 'CH11', 'CH12', 'CH13', 'CH14', 'NIVEL_ED',

    # Migración
    'CH15', 'CH15_COD', 'CH16', 'CH16_COD',

    # Condición de actividad
    'ESTADO', 'CAT_OCUP', 'CAT_INAC', 'IMPUTA',

    # Búsqueda de empleo
    'PP02C1', 'PP02C2', 'PP02C3', 'PP02C4', 'PP02C5', 'PP02C6',
    'PP02C7', 'PP02C8', 'PP02E', 'PP02H', 'PP02I',

    # Ocupación actual
    'PP03C', 'PP03D', 'PP3E_TOT', 'PP3F_TOT',
    'PP03G', 'PP03H', 'PP03I', 'PP03J',
    'INTENSI',

    # Información del empleo actual
    'PP04A', 'PP04B_COD', 'PP04D_COD', 'PP04G',
    'PP05B2_MES', 'PP05B2_ANO', 'PP05C_1', 'PP05C_2', 'PP05C_3', 'PP05F',

    # Condiciones laborales (asalariados)
    'PP07A', 'PP07C', 'PP07D', 'PP07E', 'PP07F1', 'PP07F2', 'PP07F3',
    'PP07F4', 'PP07F5', 'PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59',
    'PP07H', 'PP07I', 'PP07J', 'PP07K',

    # Ingresos laborales y totales
    'P21', 'TOT_P12', 'P47T',
    'DECINDR', 'IDECINDR', 'RDECINDR', 'GDECINDR', 'PDECINDR', 'ADECINDR',
    'PONDIIO', 'PONDII',

    # Ingresos no laborales
    'V2_M', 'V3_M', 'V4_M', 'V5_M', 'V8_M', 'V9_M', 'V10_M', 'V11_M',
    'V12_M', 'V18_M', 'V19_AM', 'V21_M', 'T_VI',

    # Ingresos familiares desde individuo (puede usarse para agregar)
    'ITF', 'IPCF', 'DECIFR', 'DECCFR', 'PONDIH'
]

# Crear el nuevo DataFrame filtrado con las columnas deseadas
df_individuo_interes = bases_individuales[clave][variables_individuo_interes].copy()

# Mostrar primeras filas del DataFrame de interés
df_individuo_interes.head()

,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,REGION,MAS_500,AGLOMERADO,PONDERA,PONDIH,...,V12_M,V18_M,V19_AM,V21_M,T_VI,ITF,IPCF,DECIFR,DECCFR,PONDIH
0,TQRMNOSUWHLMKQCDEFNFF00852219,2023,4,1,1,41,N,7,370,0,...,0,0,0,0,0,0,0,12,12,0
1,TQRMNOSUWHLMKQCDEFNFF00852219,2023,4,1,2,41,N,7,370,0,...,0,0,0,0,0,0,0,12,12,0
2,TQRMNOTXQHKOLMCDEGNFJ00794590,2023,4,1,1,44,N,17,281,387,...,0,0,0,0,0,650000,"216666,67",9,8,387
3,TQRMNOTXQHKOLMCDEGNFJ00794590,2023,4,1,2,44,N,17,281,387,...,0,0,0,0,0,650000,"216666,67",9,8,387
4,TQRMNOTXQHKOLMCDEGNFJ00794590,2023,4,1,3,44,N,17,281,387,...,0,0,0,0,0,650000,"216666,67",9,8,387


###3.2 Conformacion base de interes

In [30]:
# Asegurar tipos numéricos
df_individuo_interes['CH06'] = pd.to_numeric(df_individuo_interes['CH06'], errors='coerce')
df_individuo_interes['CH14'] = pd.to_numeric(df_individuo_interes['CH14'], errors='coerce')

# Calcular escolaridad esperada (máximo 15 años desde los 5 años de edad)
df_individuo_interes['escolaridad_esperada'] = (df_individuo_interes['CH06'] - 5).clip(lower=0, upper=15)
df_individuo_interes['escolaridad_completada'] = df_individuo_interes['CH14']

# Índice de Escolaridad Ajustado por Edad (IEAE)
df_individuo_interes['IEAE'] = df_individuo_interes['escolaridad_completada'] / df_individuo_interes['escolaridad_esperada']
df_individuo_interes.loc[df_individuo_interes['escolaridad_esperada'] == 0, 'IEAE'] = pd.NA

# Filtrar individuos válidos para IEAE
df_IEAE_validos = df_individuo_interes[(df_individuo_interes['CH06'] > 6) & (df_individuo_interes['IEAE'].notnull())]

# Calcular NE por hogar (media simple)
df_NE = df_IEAE_validos.groupby(['CODUSU', 'NRO_HOGAR'], as_index=False).agg(
    NE=('IEAE', 'mean'),
    N=('COMPONENTE', 'count')
)

# Unir NE al DataFrame de hogares
df_hogar_interes = df_hogar_interes.merge(df_NE, on=['CODUSU', 'NRO_HOGAR'], how='left')

# Crear variable de hacinamiento: personas por cuarto
df_hogar_interes['hacinamiento'] = df_hogar_interes.apply(
    lambda row: row['IX_TOT'] / row['IV2'] if pd.notnull(row['IX_TOT']) and pd.notnull(row['IV2']) and row['IV2'] > 0 else np.nan,
    axis=1
)


# Crear variables binarias para proporciones
df_individuo_interes['es_mujer'] = df_individuo_interes['CH04'] == 2
df_individuo_interes['es_analfabeto'] = df_individuo_interes['CH09'] == 2
df_individuo_interes['es_desocupado'] = df_individuo_interes['ESTADO'] == 2

# Agregar proporciones por hogar (sin ponderar)
df_agregado = df_individuo_interes.groupby(['CODUSU', 'NRO_HOGAR'], as_index=False).agg(
    proporcion_mujeres=('es_mujer', 'mean'),
    promedio_edad=('CH06', 'mean'),
    proporcion_analfabetismo=('es_analfabeto', 'mean'),
    proporcion_desocupados=('es_desocupado', 'mean')
)

# Unir agregados al DataFrame de hogares
df_hogar_interes = df_hogar_interes.merge(df_agregado, on=['CODUSU', 'NRO_HOGAR'], how='left')

# Guardar base procesada
df_hogar_interes.to_csv("hogares_2023T4.csv", index=False)
df_hogar_interes.to_excel("hogares_2023T4.xlsx", index=False)

print(df_hogar_interes.columns.tolist())

['CODUSU', 'NRO_HOGAR', 'ANO4', 'TRIMESTRE', 'REGION', 'MAS_500', 'AGLOMERADO', 'PONDERA', 'PONDIH', 'IX_TOT', 'IX_MEN10', 'IX_MAYEQ10', 'ITF', 'IPCF', 'DECIFR', 'IDECIFR', 'RDECIFR', 'GDECIFR', 'PDECIFR', 'ADECIFR', 'DECCFR', 'IDECCFR', 'RDECCFR', 'GDECCFR', 'PDECCFR', 'ADECCFR', 'IV1', 'IV2', 'IV3', 'IV4', 'IV5', 'IV6', 'IV7', 'IV8', 'IV9', 'IV10', 'IV11', 'IV12_1', 'IV12_2', 'IV12_3', 'II1', 'II2', 'II3', 'II3_1', 'II4_1', 'II4_2', 'II4_3', 'II5', 'II5_1', 'II6', 'II6_1', 'II7', 'II8', 'II9', 'V1', 'V2', 'V21', 'V22', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19_A', 'V19_B', 'VII1_1', 'VII1_2', 'VII2_1', 'VII2_2', 'VII2_3', 'VII2_4', 'NE', 'N', 'hacinamiento', 'proporcion_mujeres', 'promedio_edad', 'proporcion_analfabetismo', 'proporcion_desocupados']


##4. Regresion
$${NE}_i = f(X_i) + e_i$$

###4.1 MCO con seleccion de variables

In [31]:
# Preparar datos para la regresión

# Filtrar hogares con NE definido
df_modelo = df_hogar_interes[df_hogar_interes['NE'].notnull()].copy()

# Variables explicativas (X)
X = df_modelo[[
    'hacinamiento',
    'ITF', 'IPCF',
    'proporcion_mujeres',
    'promedio_edad',
    'proporcion_analfabetismo',
    'proporcion_desocupados'
]].copy()

# Limpiar comas decimales y convertir a float
for col in X.columns:
    X[col] = X[col].astype(str).str.replace(",", ".", regex=False)
    X[col] = pd.to_numeric(X[col], errors="coerce")

# Variable dependiente
y = df_modelo['NE']

# Eliminar filas con valores faltantes
df_modelo_limpio = pd.concat([X, y], axis=1).dropna()
X = df_modelo_limpio.drop(columns='NE')
y = df_modelo_limpio['NE']

In [32]:
# Crear y entrenar modelo con Elastic Net

# Separar conjunto de entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar pipeline con estandarización + ElasticNetCV
modelo = make_pipeline(
    StandardScaler(),
    ElasticNetCV(cv=5, l1_ratio=[.1, .5, .7, .9, .95, 1], random_state=42)
)

modelo.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('elasticnetcv',
                 ElasticNetCV(cv=5, l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 1],
                              random_state=42))])

In [33]:
# Evaluar el modelo

y_pred = modelo.predict(X_test)
print("R²:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# Mostrar coeficientes
coefs = modelo.named_steps['elasticnetcv'].coef_
for var, coef in zip(X.columns, coefs):
    print(f"{var}: {coef:.4f}")

R²: 0.02262222546987691
RMSE: 1.2259724249281607
hacinamiento: 0.0464
ITF: 0.0338
IPCF: -0.0676
proporcion_mujeres: -0.0038
promedio_edad: 0.0609
proporcion_analfabetismo: 0.1570
proporcion_desocupados: 0.0073


In [34]:
# Mostrar los coeficientes seleccionados

# Obtener coeficientes del modelo entrenado
coefs = modelo.named_steps['elasticnetcv'].coef_
variables = X.columns

print("Coeficientes del modelo (Elastic Net):")
for var, coef in zip(variables, coefs):
    print(f"{var}: {coef:.4f}")

Coeficientes del modelo (Elastic Net):
hacinamiento: 0.0464
ITF: 0.0338
IPCF: -0.0676
proporcion_mujeres: -0.0038
promedio_edad: 0.0609
proporcion_analfabetismo: 0.1570
proporcion_desocupados: 0.0073


###4.2 Bosques aleatorios

In [35]:
# Preparar datos

# Filtrar hogares con NE definido
df_modelo = df_hogar_interes[df_hogar_interes['NE'].notnull()].copy()

# Variables independientes (X)
X = df_modelo[[
    'hacinamiento',
    'ITF', 'IPCF',
    'proporcion_mujeres',
    'promedio_edad',
    'proporcion_analfabetismo',
    'proporcion_desocupados'
]].copy()

# Convertir comas a puntos y asegurar tipo float
for col in X.columns:
    X[col] = X[col].astype(str).str.replace(",", ".", regex=False)
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Variable dependiente
y = df_modelo['NE']

# Eliminar filas con valores faltantes
df_modelo_limpio = pd.concat([X, y], axis=1).dropna()
X = df_modelo_limpio.drop(columns='NE')
y = df_modelo_limpio['NE']

In [36]:
# Entrenar modelo

# Dividir datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y ajustar el modelo
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [37]:
# Evaluar modelo

# Predicciones
y_pred = rf_model.predict(X_test)

# Métricas
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R² del modelo: {r2:.4f}")
print(f"RMSE del modelo: {rmse:.4f}")

R² del modelo: -0.2596
RMSE del modelo: 1.3917


In [38]:
# Relevancia de variables

# Mostrar importancia de variables
importancia = rf_model.feature_importances_

print("\nRelevancia de variables:")
for var, imp in sorted(zip(X.columns, importancia), key=lambda x: x[1], reverse=True):
    print(f"{var}: {imp:.4f}")


Relevancia de variables:
promedio_edad: 0.3461
ITF: 0.1811
IPCF: 0.1673
hacinamiento: 0.1344
proporcion_mujeres: 0.0972
proporcion_analfabetismo: 0.0464
proporcion_desocupados: 0.0274


###4.3 Support Vector Regressor (SVR)

In [39]:
# Preparar datos

# Filtrar hogares con NE definido
df_modelo = df_hogar_interes[df_hogar_interes['NE'].notnull()].copy()

# Variables independientes (X)
X = df_modelo[[
    'hacinamiento',
    'ITF', 'IPCF',
    'proporcion_mujeres',
    'promedio_edad',
    'proporcion_analfabetismo',
    'proporcion_desocupados'
]].copy()

# Corregir comas y convertir a float
for col in X.columns:
    X[col] = X[col].astype(str).str.replace(",", ".", regex=False)
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Variable dependiente
y = df_modelo['NE']

# Eliminar filas con valores faltantes
df_modelo_limpio = pd.concat([X, y], axis=1).dropna()
X = df_modelo_limpio.drop(columns='NE')
y = df_modelo_limpio['NE']

In [40]:
# Entrenar modelo

# Dividir en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear pipeline: escalado + SVR
svr_model = make_pipeline(
    StandardScaler(),
    SVR(kernel='rbf', C=1.0, epsilon=0.1)
)

# Entrenar modelo
svr_model.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svr', SVR())])

In [41]:
# Evaluar modelo

# Predicciones
y_pred = svr_model.predict(X_test)

# Métricas
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R² del modelo SVR: {r2:.4f}")
print(f"RMSE del modelo SVR: {rmse:.4f}")

R² del modelo SVR: -0.0291
RMSE del modelo SVR: 1.2580


##5. Clasificacion
$$PE_i = \mathbb{I}({NE}_i<\tau)$$

$\tau$: Mediana

###5.1 Bosques aleatorios

In [42]:
# Crear variable PE_i

# Usar hogares con NE definido
df_clasificacion = df_hogar_interes[df_hogar_interes['NE'].notnull()].copy()

# Calcular mediana de NE
tau = df_clasificacion['NE'].median()

# Crear variable binaria: 1 si NE < mediana (pobreza educativa)
df_clasificacion['PE_i'] = (df_clasificacion['NE'] < tau).astype(int)

In [43]:
# Preparar datos

# Variables explicativas
X = df_clasificacion[[
    'hacinamiento',
    'ITF', 'IPCF',
    'proporcion_mujeres',
    'promedio_edad',
    'proporcion_analfabetismo',
    'proporcion_desocupados'
]].copy()

# Corregir comas y convertir a float
for col in X.columns:
    X[col] = X[col].astype(str).str.replace(",", ".", regex=False)
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Variable dependiente
y = df_clasificacion['PE_i']

# Eliminar filas con valores faltantes
df_clasificacion_limpio = pd.concat([X, y], axis=1).dropna()
X = df_clasificacion_limpio.drop(columns='PE_i')
y = df_clasificacion_limpio['PE_i']

In [44]:
# Entrenar modelo

# Separar en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar modelo
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [45]:
# Evaluar modelo

# Predicciones
y_pred = clf.predict(X_test)

# Métricas de evaluación
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.6179775280898876

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.62      0.63      0.63      1124
           1       0.62      0.60      0.61      1101

    accuracy                           0.62      2225
   macro avg       0.62      0.62      0.62      2225
weighted avg       0.62      0.62      0.62      2225


Matriz de confusión:
 [[710 414]
 [436 665]]


In [46]:
# Relevancia de variables

print("\nRelevancia de variables:")
for var, imp in sorted(zip(X.columns, clf.feature_importances_), key=lambda x: x[1], reverse=True):
    print(f"{var}: {imp:.4f}")


Relevancia de variables:
promedio_edad: 0.3618
IPCF: 0.1860
ITF: 0.1820
hacinamiento: 0.1253
proporcion_mujeres: 0.0908
proporcion_analfabetismo: 0.0334
proporcion_desocupados: 0.0208


###5.2 Support Vector Machine (SVM)

In [47]:
# Entrenar el modelo

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear pipeline con estandarización y SVM (con kernel RBF)
svm_model = make_pipeline(
    StandardScaler(),
    SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
)

# Entrenar
svm_model.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(random_state=42))])

In [48]:
# Evaluar el modelo

# Predicciones
y_pred = svm_model.predict(X_test)

# Métricas
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.6629213483146067

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.67      0.67      0.67      1124
           1       0.66      0.66      0.66      1101

    accuracy                           0.66      2225
   macro avg       0.66      0.66      0.66      2225
weighted avg       0.66      0.66      0.66      2225


Matriz de confusión:
 [[748 376]
 [374 727]]
